In [27]:
import numpy as np
import pandas as pd


In [24]:
users = ["User1", "User2", "User3", "User4", "User5"]
items = ["Item A", "Item B", "Item C", "Item D", "Item E"]

datasets = [
    [1, 0, 1, 1, 0],
    [1, 0, 0, 1, 1],
    [1, 0, 1, 0, 0],
    [0, 1, 0, 1, 1],
    [1, 1, 1, 0, 1],
]

df = pd.DataFrame(datasets, columns=items, index=users)
df

,Item A,Item B,Item C,Item D,Item E
User1,1,0,1,1,0
User2,1,0,0,1,1
User3,1,0,1,0,0
User4,0,1,0,1,1
User5,1,1,1,0,1


In [3]:
from sklearnex import patch_sklearn
from sklearn.metrics import jaccard_score  # 直接计算某两项的杰卡德相似系数

patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
# 计算 Item A 和 Item B 的杰卡德相似度
jaccard_score(df["Item A"], df["Item B"])

0.2

In [7]:
from sklearn.metrics import pairwise_distances  # 计算所有数据的两两杰卡德相似系数

# 计算用户间相似度
user_similar = 1 - pairwise_distances(df, metric="jaccard")  # 杰卡德相似等于 1 - 杰卡德距离
user_similar = pd.DataFrame(user_similar, columns=users, index=users)
print("用户之间的两两相似度：")
print(user_similar)

用户之间的两两相似度：
          User1  User2     User3  User4  User5
User1  1.000000   0.50  0.666667    0.2    0.4
User2  0.500000   1.00  0.250000    0.5    0.4
User3  0.666667   0.25  1.000000    0.0    0.5
User4  0.200000   0.50  0.000000    1.0    0.4
User5  0.400000   0.40  0.500000    0.4    1.0


C:\Users\Ryan\mambaforge\envs\py310\lib\site-packages\daal4py\sklearn\utils\validation.py:264: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any():
C:\Users\Ryan\mambaforge\envs\py310\lib\site-packages\daal4py\sklearn\metrics\_pairwise.py:207: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [8]:
# 计算物品间相似度
item_similar = 1 - pairwise_distances(df.T, metric="jaccard")
item_similar = pd.DataFrame(item_similar, columns=items, index=items)
print("物品之间的两两相似度：")
print(item_similar)

物品之间的两两相似度：
        Item A    Item B  Item C  Item D    Item E
Item A    1.00  0.200000    0.75    0.40  0.400000
Item B    0.20  1.000000    0.25    0.25  0.666667
Item C    0.75  0.250000    1.00    0.20  0.200000
Item D    0.40  0.250000    0.20    1.00  0.500000
Item E    0.40  0.666667    0.20    0.50  1.000000


C:\Users\Ryan\mambaforge\envs\py310\lib\site-packages\daal4py\sklearn\utils\validation.py:264: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any():
C:\Users\Ryan\mambaforge\envs\py310\lib\site-packages\daal4py\sklearn\metrics\_pairwise.py:207: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [44]:
# 开始进行过滤
# 为每一个用户找到最相似的2个用户
topN_users = {}
for i in user_similar.index:
    # 取出每一列数据，并删除自身，然后排序数据
    _df: pd.DataFrame = user_similar.loc[i].drop([i])
    _df_stored = _df.sort_values(ascending=False)

    top2 = list(_df_stored.index[:2])
    topN_users[i] = top2

print("Top2相似用户：")
print(topN_users)

Top2相似用户：
{'User1': ['User3', 'User2'], 'User2': ['User1', 'User4'], 'User3': ['User1', 'User5'], 'User4': ['User2', 'User5'], 'User5': ['User3', 'User1']}


In [29]:
# 根据 topN 的相似用户构建推荐结果
rs_results = {}
for user, sim_users in topN_users.items():
    rs_result = set()  # 存储推荐结果
    for sim_user in sim_users:
        # 构建初始的推荐结果
        rs_result = rs_result.union(set(df.loc[sim_user].replace(0, np.nan).dropna().index))
    # 过滤掉已经购买过的物品
    rs_result -= set(df.loc[user].replace(0, np.nan).dropna().index)
    rs_results[user] = rs_result
print("最终推荐结果：")
print(rs_results)

最终推荐结果：
{'User1': {'Item E'}, 'User2': {'Item B', 'Item C'}, 'User3': {'Item B', 'Item E', 'Item D'}, 'User4': {'Item A', 'Item C'}, 'User5': {'Item D'}}


In [46]:
# 为每一个用户找到最相似的2个物品
topN_items = {}
# 遍历每一行数据
for i in item_similar.index:
    # 取出每一列数据，并删除自身，然后排序数据
    _df = item_similar.loc[i].drop([i])
    print(_df)
    _df_stored = _df.sort_values(ascending=False)

    top2 = list(_df_stored.index[:2])
    topN_items[i] = top2

print("Top2相似物品：")
print(topN_items)

Item B    0.20
Item C    0.75
Item D    0.40
Item E    0.40
Name: Item A, dtype: float64
Item A    0.200000
Item C    0.250000
Item D    0.250000
Item E    0.666667
Name: Item B, dtype: float64
Item A    0.75
Item B    0.25
Item D    0.20
Item E    0.20
Name: Item C, dtype: float64
Item A    0.40
Item B    0.25
Item C    0.20
Item E    0.50
Name: Item D, dtype: float64
Item A    0.400000
Item B    0.666667
Item C    0.200000
Item D    0.500000
Name: Item E, dtype: float64
Top2相似物品：
{'Item A': ['Item C', 'Item D'], 'Item B': ['Item E', 'Item C'], 'Item C': ['Item A', 'Item B'], 'Item D': ['Item E', 'Item A'], 'Item E': ['Item B', 'Item D']}


In [47]:
rs_results = {}
# 构建推荐结果
for user in df.index:  # 遍历所有用户
    rs_result = set()
    for item in df.loc[user].replace(0, np.nan).dropna().index:  # 取出每个用户当前已购物品列表
        rs_result = rs_result.union(topN_items[item])
    # 过滤掉用户已购的物品
    rs_result -= set(df.loc[user].replace(0, np.nan).dropna().index)
    # 添加到结果中
    rs_results[user] = sorted(rs_result, reverse=True)

print("最终推荐结果：")
print(rs_results)

最终推荐结果：
{'User1': ['Item E', 'Item B'], 'User2': ['Item C', 'Item B'], 'User3': ['Item D', 'Item B'], 'User4': ['Item C', 'Item A'], 'User5': ['Item D']}
